In [68]:
import json
import os
import pandas as pd
from pymongo import MongoClient

In [78]:
# Define the folder path where your JSON files are located
folder_path = "Resources"

# Read the "IMDB_API_adventure100_location_geoapify.json" data
geoapify_file_path = os.path.join(folder_path, "IMDB_API_adventure100_location_geoapify.json")
with open(geoapify_file_path, 'r') as geoapify_file:
    geoapify_data = json.load(geoapify_file)

# Read the "IMDB_API_adventure100_location.json" data
location_file_path = os.path.join(folder_path, "IMDB_API_adventure100_location.json")
with open(location_file_path, 'r') as location_file:
    location_data = json.load(location_file)

# Create a combined JSON data with the desired fields
combined_data = []
for geoapify_entry, location_entry in zip(geoapify_data, location_data):
    combined_entry = {
        "id": location_entry.get("id", None),
        "location_name": geoapify_entry.get("location_name", None),
        "lat": geoapify_entry.get("lat", None),
        "lon": geoapify_entry.get("lon", None),
        "location_id": location_entry.get("location_id", None)
    }
    combined_data.append(combined_entry)

# Name of the combined JSON file
combined_file_path = 'Resources/IMDB_combined_location_data.json'

# Save the combined data to a single JSON file
with open(combined_file_path, 'w') as combined_file:
    json.dump(combined_data, combined_file)


In [82]:
# Convert the combined data to a pandas DataFrame
df = pd.DataFrame(combined_data)

# Define the path for the CSV file
csv_file_path = 'Resources/IMDB_combined_location_data.csv'

# Save the DataFrame as a CSV file
df.to_csv(csv_file_path, index=False)

In [80]:
# Define the folder path where your JSON files are located
folder_path = "Resources"

# Define a dictionary to store the combined data
movie_data = {}

# List of JSON files with their corresponding fields
json_files = [
    {
        "file": "IMDB_API_adventure100_moiveinfo.json",
        "fields": ["id", "image_url", "runningTimeInMinutes", "title", "titleType", "year", "genres", "releaseDate"]
    },
    {
        "file": "IMDB_combined_location_data.json",
        "fields": ["id", "location_name", "lat", "lon", "location_id"]
    },
    {
        "file": "IMDB_API_adventure100_rating.json",
        "fields": ["id", "canRate", "rating", "ratingCount", "topRank", "bottomRank", "ratingsHistograms"]
    },
    {
        "file": "IMDB_API_adventure100_soundtrack.json",
        "fields": ["id", "soundtracks_id", "soundtracks_name", "soundtracks_comment"]
    }
]

# Iterate through the JSON files and organize the data
for json_file_info in json_files:
    file_path = os.path.join(folder_path, json_file_info["file"])
    with open(file_path, 'r') as json_file:
        data = json.load(json_file)
        for entry in data:
            movie_id = entry.get("id")
            if movie_id not in movie_data:
                movie_data[movie_id] = {}
            for field in json_file_info["fields"]:
                movie_data[movie_id][field] = entry.get(field, None)


# Name of the combined JSON file
combined_file_path = 'Resources/combined_data.json'

# Save the combined data to a single JSON file
with open(combined_file_path, 'w') as combined_file:
    json.dump(list(movie_data.values()), combined_file)

In [81]:
# Load the combined JSON file into MongoDB
client = MongoClient('localhost', 27017)  
db = client['IMDB_MOVIES'] 
collection = db['movies']

# Load the data into MongoDB
with open(combined_file_path, 'r') as combined_file:
    data = json.load(combined_file)
    collection.insert_many(data)

print(f'Movie data loaded into MongoDB collection {collection.name}')

Movie data loaded into MongoDB collection movies
